In [1]:
import os
import glob
import numpy as np
import xarray as xr
from obspy.signal.filter import bandpass


from tools_pcc import *
from stack_functions_new import *
from xdas.io.febus import read as read_das
from xdas.io.febus import correct_gps_time

In [2]:
import multiprocessing as mp

In [3]:
user = "ipgp"

if user == "ipgp" :
    path1 = "/media/hugonnet/DISKENOIR/fichiers-nc/"
    path2 = "/media/hugonnet/DISKENOIR/Correlograms/Cross-correlation/" # storage of correlograms for cross-correlation
    path3 = "/media/hugonnet/DISKENOIR/Correlograms/Auto-correlation/" # storage of correlograms for auto-correlation
    path4 = "/home/hugonnet/Documents/DAS-Lucie/Codes/Sergi-ts-PWS/src/" # location of ts_pws file

elif user == "mba" :
    path1 = "/Volumes/DISKENOIR/fichiers-nc/"
    path2 = "/Volumes/DISKENOIR/Correlograms/Cross-correlation/" # storage of correlograms for cross-correlation
    path3 = "/Volumes/DISKENOIR/Correlograms/Auto-correlation/" # storage of correlograms for auto-correlation
    path4 = "/Users/lucie/Dropbox/MINES_3A/Stage-IPGP/DAS/Codes/Sergi-ts-PWS/src/" # location of ts_pws file

elif user == "eleonore" :
    path1 = '/Users/stutz/DAS/DAS_STROMBOLI/2022_STROMBOLI_MANIP/SILIXA_iDAS/DATA_VLP_FEBUS/'

In [4]:
files = glob.glob(path1 + '*.nc')
size_files = len(files)
files.sort()

In [5]:
files

['/media/hugonnet/DISKENOIR/fichiers-nc/SR_FEBUS_STROMBOLI_2022-10-25_00-10-20_UTC.nc',
 '/media/hugonnet/DISKENOIR/fichiers-nc/SR_FEBUS_STROMBOLI_2022-10-25_01-10-20_UTC.nc',
 '/media/hugonnet/DISKENOIR/fichiers-nc/SR_FEBUS_STROMBOLI_2022-10-25_02-10-20_UTC.nc',
 '/media/hugonnet/DISKENOIR/fichiers-nc/SR_FEBUS_STROMBOLI_2022-10-25_03-10-20_UTC.nc',
 '/media/hugonnet/DISKENOIR/fichiers-nc/SR_FEBUS_STROMBOLI_2022-10-25_04-10-20_UTC.nc',
 '/media/hugonnet/DISKENOIR/fichiers-nc/SR_FEBUS_STROMBOLI_2022-10-25_05-10-20_UTC.nc',
 '/media/hugonnet/DISKENOIR/fichiers-nc/SR_FEBUS_STROMBOLI_2022-10-25_06-10-20_UTC.nc',
 '/media/hugonnet/DISKENOIR/fichiers-nc/SR_FEBUS_STROMBOLI_2022-10-25_07-10-20_UTC.nc',
 '/media/hugonnet/DISKENOIR/fichiers-nc/SR_FEBUS_STROMBOLI_2022-10-25_08-10-20_UTC.nc',
 '/media/hugonnet/DISKENOIR/fichiers-nc/SR_FEBUS_STROMBOLI_2022-10-25_09-10-20_UTC.nc',
 '/media/hugonnet/DISKENOIR/fichiers-nc/SR_FEBUS_STROMBOLI_2022-10-25_10-10-20_UTC.nc',
 '/media/hugonnet/DISKENOIR/fich

In [6]:
corr_type = 'phase-cross'   # can be 'phase-cross' (cross-correlation), 'phase-auto' (auto-correlation), 'normal', or 'onebit'

# define variable for correlation
window = 300 			# duration of time window in second
lag = window/2 			# shift of starting time on the next iteration
time_ini = 0 			# time initialization

# filter boundary (Hz)
lowcut = 1
highcut = 10

start_position = 'mid_all' 

# define which part of fiber is used - not used when start_position is not 'mid_all'
start_channel = 56
end_channel = 430

ref_channel = 350		# reference point for correlation. currently used: 100, 200, 300, 400, 500, ... (depends on the part of the fiber that is being used)
nb_channel = 150		# number of channel used - not used when start_position is not 'mid_all'

# gauge length parameters
change_gauge_length = False
gauge_length = 50

In [12]:
def task_cc(file) :

	#print(f'Processing file {i} out of {size_files}')
	
	if corr_type == 'phase-cross' :
		folder_destination = file.replace(path1, path2).replace(".nc", "/")

	elif corr_type == 'phase-auto' :
		folder_destination = file.replace(path1, path3).replace(".nc", "/")
		
		
	if not os.path.exists(folder_destination):
		os.mkdir(folder_destination)


	ds = xr.open_dataset(file)
	channels_coord = ds['channels']

	iDas_nc = xr.DataArray( 
		data=ds['strain_rates'],  # Remplacez par les données réelles de 'strain_rates'
		dims=("times", "channels"),  # Spécifiez les dimensions 
		coords={"times": ds['strain_rates'].coords["times"], "channels": channels_coord}
		)
    
	dt = ( iDas_nc['times'][1].item() - iDas_nc['times'][0].item() ) * 1e-9 # s
	fs = 1/dt # Hz

	f = open('error.txt','w') # files with few channels are not used and are stored into the file 'f'

	if iDas_nc.shape[1] < 463: 
		print(f'This file only has {iDas_nc.shape[1]} channels. Skipping this file')
		f.write(f'{iDas_nc.shape[1]} channels\n')
		# skipping file if the channel is not complefind . -type f -path "*/C*/" -exec mv {} DossierDestination \;te

	else :

		print(f'Filtering using bandpass filter between {lowcut} and {highcut} Hz ...')
		
		if change_gauge_length == False:
			traces_filt = iDas_nc.copy()
			print('no gauge length change')
			
		else:
        
        	# to try GL = 10
			traces_filt = iDas_nc.copy()
			traces_filt[:,0] = iDas_nc[:,0] + iDas_nc[:,1] # the first channel with gl = 10 is defined as channel 0 + channel 1
			traces_filt[:,-1] = iDas_nc[:,-2] + iDas_nc[:,-1] # the last channel with gl = 10 is defined as channel N-1 + channel N
			print(f'Converting data to using gauge length = 10')
        	#for i in range(1,traces_filt.shape[1]-2): # each end is skipped because it has been calculated in the previous line
			for i_channel in range(57, iDas_nc['channels'].values[-2]+1):
				traces_filt[:,i_channel] = iDas_nc[:,i_channel-1] + iDas_nc[:,i_channel] + iDas_nc[:,i_channel+1]
			traces_filt_gl10 = traces_filt[::2]


    	# for j in [56,3249]:
		for j in range (start_channel,end_channel+1):
			try:
		    	#traces_filt[:,j] = filter_bandpass(traces_filt[:,j], lowcut, highcut, fs,order=4) # filter defined in stack_functions and using scipy.signal.butter filter
				traces_filt[:,j] = bandpass(traces_filt[:,j],lowcut,highcut,fs, zerophase=True) # obspy butterworth filter (faster)
			except:
				print('filtrage pas abouti pour j = ', j)
				#None
		
		length = iDas_nc.shape[0]	# total length of data

		# cross correlate

		if corr_type == 'phase-cross' :
			print('\nStarting the phase cross-correlation  ...')
			pcc, t = cross_correlate(traces_filt,folder_destination,window,lag,length,time_ini,start_channel,end_channel,nb_channel,
                	                      ref_channel,start_position,onebit=False,phase_cc=True,decimate=1,resample=1)
			
		elif corr_type == 'phase-auto' :
			print('\nStarting the phase auto-correlation  ...')
			pcc_auto, t = auto_correlate(traces_filt,foresult = tqdm(der_destination,window,lag,length,time_ini,start_channel,end_channel,nb_channel,
            	                          ref_channel,start_position,onebit=False,phase_cc=True,decimate=1,resample=1)


		#print('\nStarting the normal cross-correlation  ...')
		#cc_normal, t = cross_correlate(traces_filt,window,lag,length,time_ini,start_channel,end_channel,nb_channel,
		#		ref_channel,start_position,onebit=False,phase_cc=False,decimate=decimate,resample=resample)

		#cc_onebit, t = cross_correlate(traces_filt,window,lag,length,time_ini,start_channel,end_channel,nb_channel,
		#		ref_channel,start_position,onebit=True,phase_cc=False,decimate=decimate,resample=resample)


	f.close()


In [13]:
mp.cpu_count()

16

Pool for cross-correlation calculation:

In [15]:
if __name__ == '__main__' :

    #mp.set_start_method('spawn')
    n_processes = 8

    with mp.Pool(processes = n_processes) as pool :
        pool.map(task_cc, files)

Filtering using bandpass filter between 1 and 10 Hz ...
no gauge length change

Starting the phase cross-correlation  ...
Start of the 1th time window. Starttime = 0.0 s
Start of the 2th time window. Starttime = 150.0 s
Filtering using bandpass filter between 1 and 10 Hz ...
no gauge length change
Start of the 3th time window. Starttime = 300.0 s
Filtering using bandpass filter between 1 and 10 Hz ...
no gauge length change

Starting the phase cross-correlation  ...
Start of the 1th time window. Starttime = 0.0 s

Starting the phase cross-correlation  ...
Start of the 1th time window. Starttime = 0.0 s
Start of the 4th time window. Starttime = 450.0 s
Start of the 2th time window. Starttime = 150.0 s
Start of the 2th time window. Starttime = 150.0 s
Start of the 5th time window. Starttime = 600.0 s
Start of the 3th time window. Starttime = 300.0 s
Start of the 3th time window. Starttime = 300.0 s
Start of the 6th time window. Starttime = 750.0 s
Start of the 4th time window. Starttime 

Pool for copying cc files:

In [7]:
numbers = list(range(56, 100))
numbers

os.chdir(path2)

In [8]:
def task_cp(i_channel) :
    
    destination = path2 + f'all_correlograms_{ref_channel}_{i_channel}/'
    print(destination + '/n')
    
    if not os.path.exists(destination) : 
        os.mkdir(destination)

    os.system(f'find . -type f -wholename "*/Correlogram_{ref_channel}_{i_channel}_pcc/{ref_channel}_{i_channel}_*.sac" -exec cp {{}} {destination} \;')


In [9]:
if __name__ == '__main__' :

    n_processes = 10

    with mp.Pool(processes = n_processes) as pool :
        pool.map(task_cp, numbers)

/media/hugonnet/DISKENOIR/Correlograms/Cross-correlation/all_correlograms_350_60//n/media/hugonnet/DISKENOIR/Correlograms/Cross-correlation/all_correlograms_350_58//n/media/hugonnet/DISKENOIR/Correlograms/Cross-correlation/all_correlograms_350_56//n/media/hugonnet/DISKENOIR/Correlograms/Cross-correlation/all_correlograms_350_64//n/media/hugonnet/DISKENOIR/Correlograms/Cross-correlation/all_correlograms_350_66//n/media/hugonnet/DISKENOIR/Correlograms/Cross-correlation/all_correlograms_350_62//n/media/hugonnet/DISKENOIR/Correlograms/Cross-correlation/all_correlograms_350_68//n/media/hugonnet/DISKENOIR/Correlograms/Cross-correlation/all_correlograms_350_70//n/media/hugonnet/DISKENOIR/Correlograms/Cross-correlation/all_correlograms_350_74//n/media/hugonnet/DISKENOIR/Correlograms/Cross-correlation/all_correlograms_350_72//n









/media/hugonnet/DISKENOIR/Correlograms/Cross-correlation/all_correlograms_350_69//n
/media/hugonnet/DISKENOIR/Correlograms/Cross-correlation/all_correlograms_35

Inutile :

In [34]:
def task_cp(number) :
    if corr_type == 'phase-cross' :
        path = path2
    elif corr_type == 'phase-auto' :
        path = path3

    os.chdir(path)
    
    destination = path + f'all_correlograms_{ref_channel}_{number}/'
    print(destination)
    
    os.mkdir(destination)

    os.system(f'find . -type f -wholename "*/Correlogram_{ref_channel}_{number}_pcc/{ref_channel}_{number}_*.sac" -exec cp {{}} {destination} \;')


In [37]:
if corr_type == 'phase-cross' :
    folder = path2

elif corr_type == 'phase-auto' :
    folder = path3
        
os.system(f'cp {path4}ts_pws {folder}')
os.chdir(folder)


for i_channel in range(430,431) :   
    os.system('find . -type f -empty -print -delete') # delete empty file
    print(f'tf-pws for correlogram {ref_channel} and {i_channel}')
    os.system(f'find . -type f -wholename "*/Correlogram_{ref_channel}_{i_channel}_pcc/{ref_channel}_{i_channel}_*.sac" --color=never > tf-pws-{ref_channel}_{i_channel}.in')
    #os.system('ls *.sac > tf-pws.in')
    os.system(f'./ts_pws tf-pws-{ref_channel}_{i_channel}.in osac="tspws_pcc-{ref_channel}_{i_channel}"')
    # there will be a warning that says the header is corrupted, just ignore it since we just need the cross-correlogram time array, the header information is not important

In [ ]:
for i_channel in range(start_channel,end_channel+1):
    
    if corr_type == 'phase-cross' :
        folder = path2

    elif corr_type == 'phase-auto' :
        folder = path3
        
    os.system(f'cp {path4}ts_pws {folder}')
    os.chdir(folder)
    os.system('find . -type f -empty -print -delete') # delete empty file
    print(f'tf-pws for correlogram {ref_channel} and {i_channel}')
    os.system(f'find . -type f -wholename "*/Correlogram_{ref_channel}_{i_channel}_pcc/{ref_channel}_{i_channel}_*.sac" --color=never > tf-pws-{ref_channel}_{i_channel}.in')
    #os.system('ls *.sac > tf-pws.in')
    os.system(f'./ts_pws tf-pws-{ref_channel}_{i_channel}.in osac="tspws_pcc-{ref_channel}_{i_channel}"')
    # there will be a warning that says the header is corrupted, just ignore it since we just need the cross-correlogram time array, the header information is not important

Process : ATTENTION FAIT CRASHER

In [9]:
if __name__ == '__main__' :

    processes = [Process(target=task, args=(files[i],)) for i in range(5,20)]
    for process in processes :
        process.start()
    for process in processes :
        process.join()
    print('Done', flush=True)

Filtering using bandpass filter between 1 and 10 Hz ...
no gauge length change

Starting the phase cross-correlation  ...
Start of the 1th time window. Starttime = 0.0 s
Filtering using bandpass filter between 1 and 10 Hz ...
no gauge length change
Start of the 2th time window. Starttime = 150.0 s

Starting the phase cross-correlation  ...
Start of the 1th time window. Starttime = 0.0 s
Filtering using bandpass filter between 1 and 10 Hz ...
Start of the 3th time window. Starttime = 300.0 s
no gauge length change
Filtering using bandpass filter between 1 and 10 Hz ...
Start of the 2th time window. Starttime = 150.0 s
Start of the 4th time window. Starttime = 450.0 s

Starting the phase cross-correlation  ...
Start of the 1th time window. Starttime = 0.0 s
no gauge length change
Start of the 3th time window. Starttime = 300.0 s

Starting the phase cross-correlation  ...
Start of the 1th time window. Starttime = 0.0 s
Start of the 5th time window. Starttime = 600.0 s
Start of the 2th tim